In [0]:
# Setup Jupyter configuration for interactive mode and retina mode for anti-aliased graph rendering

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
# Importing libraries we're going to use

import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime


In [0]:
# Fetch list of countries

countries_data = pd.read_json('https://api.covid19api.com/countries').sort_values(['Country'])

In [0]:
# Declaring a dropwdown widget for selecting countries

country_dropdown = widgets.Dropdown(
    options=['Global'] + countries_data['Country'].tolist(),
    value='Global',
    description='Countries :',
    disabled=False,
)

In [0]:
# Declare toggle button to switch between daily and cumulative data 

data_button = widgets.ToggleButtons(
    options=['Cumulative', 'Daily'],
    description='Data Type :',
    button_style='success',
    icon='check'
)

In [0]:
# Declare toggle button to enable and disable grid

grid_button = widgets.ToggleButtons(
    options=["On", "Off"],
    description='Grid :',
    button_style='success',
    icon='check'
)

In [0]:
# Declare toggle button to declare graph type

graph_type = widgets.ToggleButtons(
    options=["Regular", "Uniform"],
    description='Graph Type :',
    button_style='success',
    icon='check'
)

In [0]:
# Declare plot button

button = widgets.Button(
    description='Plot',
)

In [0]:
# Padding method that returns 2% of the max data available for absoluted label padding

def padding(max):
    return int((max*2)/100)

In [0]:
# Searching methods that matches country name to it's slug value

def country_finder(location):
    loc = countries_data.loc[countries_data['Country']=='India', ['Slug']].values.tolist()[0]
    return loc[0]

In [0]:
# Global data plotting method

def plot_global(data, grid):
    response = requests.get('https://api.covid19api.com/summary')
    response.raise_for_status()
    global_data = json.loads(response.content.decode(encoding='UTF-8',errors='strict'))

    if(data=='Cumulative'):
        pre = 'Total'
    else:
        pre = 'New'
    
    global_confirmed_data = pd.DataFrame(global_data['Countries']).sort_values(['{0}Confirmed'.format(pre)],ascending=False).head(10)
    global_deaths_data = pd.DataFrame(global_data['Countries']).sort_values(['{0}Deaths'.format(pre)],ascending=False).head(10)
    global_recovered_data = pd.DataFrame(global_data['Countries']).sort_values(['{0}Recovered'.format(pre)],ascending=False).head(10)

    fig = plt.figure(figsize=(30, 20))
    fig.suptitle('Global Top 10 COVID-19 Statistics', fontsize=16)
    fig.tight_layout()

    ax1 = plt.subplot2grid((2, 3), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((2, 3), (0, 1), rowspan=2)
    ax3 = plt.subplot2grid((2, 3), (0, 2), rowspan=2)
    

    country = global_confirmed_data['Country'].tolist()
    confirmed = global_confirmed_data['{0}Confirmed'.format(pre)].tolist()
    recovered = global_recovered_data['{0}Recovered'.format(pre)].tolist()
    deaths = global_deaths_data['{0}Deaths'.format(pre)].tolist()

    ax1.set_title(pre + ' Confirmed')
    ax1.barh(np.arange(len(country)), confirmed)
    ax1.set_yticks(np.arange(0,10))
    ax1.invert_yaxis()
    for i, v in enumerate(confirmed):
        ax1.text(padding(confirmed[0]), i + .05, '{} | {}'.format(country[i],v), color='black')

    ax2.set_title(pre + ' Recovered')
    ax2.barh(np.arange(len(country)), recovered)
    ax2.set_yticks(np.arange(0,10))
    ax2.invert_yaxis()
    for i, v in enumerate(recovered):
        ax2.text(padding(recovered[0]), i + .05, '{} | {}'.format(country[i],v), color='black')

    ax3.set_title(pre + ' Deaths')
    ax3.barh(np.arange(len(country)), deaths)
    ax3.set_yticks(np.arange(0,10))
    ax3.invert_yaxis()
    for i, v in enumerate(deaths):
        ax3.text(padding(deaths[0]), i + .05, '{} | {}'.format(country[i],v), color='black')

In [0]:
# Country-specific data plotting method

def plot_country(data, grid, location, gtype):
    slug = country_finder(location)
    country_data = pd.read_json('https://api.covid19api.com/country/{}'.format(location))
    if(len(country_data) == 0):
      print('No statistics available')
      return
    date_list = country_data.groupby('Date')['Date']
    confirmed_list = country_data.groupby('Date').sum()['Confirmed'].tolist()[1:-1]
    death_list = country_data.groupby('Date').sum()['Deaths'].tolist()[1:-1]
    recovered_list = country_data.groupby('Date').sum()['Recovered'].tolist()[1:-1]

    if(data=='Cumulative'):
        pre = 'Total'
    else:
        pre = 'New'
        confirmed_list = [ confirmed_list[i] - confirmed_list[i-1] for i, _ in enumerate(confirmed_list)][1:]
        recovered_list = [ recovered_list[i] - recovered_list[i-1] for i, _ in enumerate(recovered_list)][1:]
        death_list = [ death_list[i] - death_list[i-1] for i, _ in enumerate(death_list)][1:]

    fig = plt.figure(figsize=(30, 20))
    fig.suptitle('{} COVID-19 Statistics'.format(location), fontsize=16)
    fig.tight_layout()

    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=3)
    ax2 = plt.subplot2grid((3, 3), (1, 0), colspan=3)
    ax3 = plt.subplot2grid((3, 3), (2, 0), colspan=3)

    if(gtype=="Uniform"):
        ax1.axis(ymax=confirmed_list[-1]+1000)
        ax2.axis(ymax=confirmed_list[-1]+1000)
        ax3.axis(ymax=confirmed_list[-1]+1000)
    else:
        ax1.axis(ymax=confirmed_list[-1]+1000)
        ax2.axis(ymax=recovered_list[-1]+1000)
        ax3.axis(ymax=death_list[-1]+1000)

    if(grid=="On"):
        ax1.grid(True)
        ax2.grid(True)
        ax3.grid(True)
    else:
        ax1.grid(False)
        ax2.grid(False)
        ax3.grid(False)

    
    ax1.set_title('{} Confirmed Cases'.format(pre))
    ax1.set_ylabel('No. of cases')
    ax1.set_xlabel('Days (from day 0 to present day)')
    ax1.plot(confirmed_list)
    ax1.set_xticks(np.arange(len(confirmed_list)))
    ax2.set_title('{} Recovered Cases'.format(pre))
    ax2.set_ylabel('No. of cases')
    ax2.set_xlabel('Days (from day 0 to present day)')
    ax2.plot(recovered_list)
    ax2.set_xticks(np.arange(len(recovered_list)))
    ax3.set_title('{} Death Cases'.format(pre))
    ax3.set_ylabel('No. of cases')
    ax3.set_xlabel('Days (from day 0 to present day)')
    ax3.set_xticks(np.arange(len(death_list)))
    ax3.plot(death_list)

In [0]:
# Plotting method that is executed when button is pressed

def plot(b=None):
    location = country_dropdown.value
    data = data_button.value
    grid = grid_button.value
    gtype = graph_type.value


    if(location=='Global'):
        plot_global(data, grid)
    else:
        plot_country(data, grid, location, gtype)


In [0]:
# Button on click event

@button.on_click
def plot_on_click(b):
    out.clear_output(wait=True)
    with out:
        plot()
        plt.show()

tab = VBox(children=[country_dropdown,
                     data_button,
                     graph_type,
                     grid_button,
                     ])

out = widgets.Output()
tab = widgets.Tab(children=[tab])
tab.set_title(0, 'Covid-19 Dashboard')
VBox(children=[tab, button, out])